In [1]:
import CreateSamples
import OT2_Ouzo_Commands_API2c as ot2_commands
import Ouzo_Graph_Tools as ouzo_graphs
from opentrons import simulate, execute, protocol_api
import importlib # for reloading packages
import pandas as pd

C:\Users\Edwin\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\Edwin\.opentrons\robot_settings.json not found. Loading defaults


Loading json containers...
Json container file load complete, listing database
Found 0 containers to add. Starting migration...
Database migration complete!


In [35]:
importlib.reload(CreateSamples)
importlib.reload(ot2_commands)

<module 'OT2_Ouzo_Commands_API2c' from 'C:\\Users\\Edwin\\Desktop\\OT2Protocols\\ot2protocol\\Ouzo_OT2_Sampling\\OT2_Ouzo_Commands_API2c.py'>

In [36]:
# Step 1: Call and Collect

labware_dir_path = r"C:\Users\Edwin\Desktop\OT2Protocols\ot2protocol\Ouzo_OT2_Sampling\Custom Labware"
labware_dict = ot2_commands.custom_labware_dict(labware_dir_path)
custom_labware_dict= labware_dict

# these variables although have the same names when searching for the stocks should now replace those for the actual selectede one. 
path = r"C:\Users\Edwin\Desktop\OT2Protocols\ot2protocol\Ouzo_OT2_Sampling\Testing Plans\Example_Working_Protocol.csv"
experiment_csv_dict = CreateSamples.get_experiment_plan(path)

In [37]:
# Step 2: Create wtf and vol df, combine into one as to make any filtering affect both.
wtf_sample_canidates = CreateSamples.generate_candidate_lattice_concentrations(experiment_csv_dict)
volume_sample_canidates = CreateSamples.calculate_ouzo_volumes_from_wtf(wtf_sample_canidates, experiment_csv_dict)
complete_df_uf = CreateSamples.combine_df(wtf_sample_canidates, volume_sample_canidates) # unfiltered

In [39]:
# Step 3: Apply filters through df based logic, currently 4 filters exist (volume, total, general min and general max)
complete_df_f1 = CreateSamples.pipette_volume_restriction_df(complete_df, 30, 1000, experiment_csv_dict['Volume Restriction']) # last argument is optional
complete_df_f2 = CreateSamples.total_volume_restriction_df(complete_df_f1,650)
complete_df_f3 = CreateSamples.general_max_restriction(complete_df_f2, 360, 'ethanol-PFH-stock uL')
complete_df_f3

,dppc molf,dspe-peg molf,PFH molf,ethanol molf,water molf,dppc-stock uL,dspe-peg-stock uL,ethanol-PFH-stock uL,ethanol-stock uL,water-stock uL,Total Sample Volume uL
12,0.0,0.000125,0.000000,0.631579,0.368296,0.0,87.499927,0.000000,303.574594,257.807237,648.881757
13,0.0,0.000125,0.000000,0.684211,0.315664,0.0,87.499927,0.000000,330.124518,220.965132,638.589576
14,0.0,0.000125,0.000000,0.736842,0.263033,0.0,87.499927,0.000000,355.555372,184.123026,627.178325
15,0.0,0.000125,0.000000,0.789474,0.210401,0.0,87.499927,0.000000,379.862011,147.280921,614.642859
16,0.0,0.000125,0.000000,0.842105,0.157770,0.0,87.499927,0.000000,403.053911,110.438816,600.992653
17,0.0,0.000125,0.000000,0.894737,0.105138,0.0,87.499927,0.000000,425.156764,73.596711,586.253401
18,0.0,0.000125,0.000000,0.947368,0.052507,0.0,87.499927,0.000000,446.214094,36.754605,570.468626
31,0.0,0.000125,0.000707,0.631579,0.367589,0.0,87.499927,41.992128,261.878843,257.312287,648.683185
32,0.0,0.000125,0.000707,0.684211,0.314957,0.0,87.499927,41.992128,288.428767,220.470182,638.391004
33,0.0,0.000125,0.000707,0.736842,0.262326,0.0,87.499927,41.992128,313.859621,183.628077,626.979752


In [ ]:
del final_volume_df['Total Sample Volume uL']
stock_volumes = final_volume_df.values
stock_volumes = CreateSamples.rearrange_2D_list(stock_volumes)

In [ ]:
protocol = simulate.get_protocol_api('2.0', extra_labware=labware_dict)
max_vol = ot2_commands.find_max_volume_labware(experiment_csv_dict, labware_dict)
stock_ranges = ot2_commands.stock_well_ranges(final_volume_df, max_vol) # set up volumes order
# make a function that will print the stocks to make, amount, and order in setup (maybe just order does not need to be actual stockpalte but )z

In [ ]:
protocol = simulate.get_protocol_api('2.0', extra_labware=labware_dict)
loaded_dict = ot2_commands.loading_labware(protocol, experiment_csv_dict) # the protocol above has been modified globally!

In [ ]:
info = ot2_commands.pipette_stock_volumes(protocol, loaded_dict, stock_volumes, stock_ranges)

In [ ]:
info['info concat']

In [ ]:
create.create_csv(r"C:\Users\Edwin\Desktop\testz", info['info concat'], seleted_wtfs_plus_blank, experiment_csv_dict)
df = pd.read_csv(r"C:\Users\Edwin\Desktop\test")